In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
x = torch.ones(16, 5, 5)
print(x.shape)
print(x.size())
print(x.size()[1:])
xsize = x.size()[1:]
num_features = 1
for s in xsize:
    num_features *= s
print(num_features)
x = x.view(-1, num_features)
print(x.shape)

torch.Size([16, 5, 5])
torch.Size([16, 5, 5])
torch.Size([5, 5])
25
torch.Size([16, 25])


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # kernal
        self.conv1 = nn.Conv2d(1, 6, 5)   # 1 input image channel, 6 output channels, 5*5 square convolution
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    # 根据forward可知具体结构： 
    # input(1 * 32 * 32) 
    # -> conv1(6 * 28 * 28) -> relu(6 * 28 * 28) -> pool2d(6 * 14 * 14) 
    # -> conv2(16 * 10 * 10) -> relu(16 * 10 * 10) -> pool2d(16 * 5 * 5) -> flatten(16 * 25)
    # -> fc1(120) -> relu(120)
    # -> fc2(84) -> relu(84)
    # -> fc3(10)
    def forward(self, x):   # x is the input  (32 * 32)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)  # pool2d 的第二个参数 2，即表示 (2 * 2)
        x = x.view(-1, self.num_flat_features(x))    # 展开操作，变成一列向量
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
        
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
params = list(net.parameters())
print(len(params))

10


In [5]:
for i in range(len(params)):
    print(params[i].size())
    
# 中间的一维参数是 bias 参数，因此和对应输出的通道数相等

torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [6]:
class Nettest(nn.Module):
    def __init__(self):
        super(Nettest, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, bias=False)
    
    def forward(self, x):
        x = self.conv1(x)
        return x
    
nettest = Nettest()
print(nettest)
pparams = list(nettest.parameters())
for pp in pparams:
    print(pp.size())

Nettest(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), bias=False)
)
torch.Size([6, 1, 5, 5])


In [7]:
input = torch.randn(1, 1, 32, 32)  # batch_size * channel_num * H * W (conv2d)
out = net(input)
print(out)
print(out.size())

tensor([[-0.0816,  0.1507,  0.0287,  0.0862,  0.1040, -0.0837,  0.1854,  0.0519,
         -0.0587, -0.0270]], grad_fn=<AddmmBackward>)
torch.Size([1, 10])


In [8]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [9]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.2668, grad_fn=<MseLossBackward>)


In [10]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [11]:
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0061,  0.0130, -0.0008,  0.0012, -0.0013,  0.0011])


In [12]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [13]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()